# 6 Applications to point clouds

## 6.1 Denoising

**TODO**
1. take clean point cloud
1. add ranom noise
1. denoise with low-pass filtering

## 6.2 Curvature estimation

**TODO**: estimate curvature with wavelets, <https://pygsp.readthedocs.io/en/stable/tutorials/wavelet.html>